# Ao3 Extract Storyline

### Install Python Library [Optional]

In [ ]:
pip install praw

### Import Libraries

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.parse import quote
import traceback
import os
import re

### Constants

In [5]:

headers = {"User-Agent": "Mozilla/5.0"}

# ✅ Step 1: Get fandoms from Books & Literature page
base_fandom_url_list = [
    "https://archiveofourown.org/media/Anime%20*a*%20Manga/fandoms",
    "https://archiveofourown.org/media/Books%20*a*%20Literature/fandoms",
    "https://archiveofourown.org/media/Cartoons%20*a*%20Comics%20*a*%20Graphic%20Novels/fandoms",
    "https://archiveofourown.org/media/Celebrities%20*a*%20Real%20People/fandoms",
    "https://archiveofourown.org/media/Movies/fandoms",
    "https://archiveofourown.org/media/Music%20*a*%20Bands/fandoms",
    "https://archiveofourown.org/media/Other%20Media/fandoms",
    "https://archiveofourown.org/media/Theater/fandoms",
    "https://archiveofourown.org/media/TV%20Shows/fandoms",
    "https://archiveofourown.org/media/Video%20Games/fandoms",
    "https://archiveofourown.org/media/Uncategorized%20Fandoms/fandoms"
]
fileName = "ao3_books_fandom_stories.csv"


### AO3 (Archive of Our Own) Scraper

In [10]:
fandom_links = []

for base_fandom_url in base_fandom_url_list:
    for page in range(1, 2):  # Try first 2 pages
        soap=''
        try:
            url = base_fandom_url.format(page)
            print(f"Fetching fandoms from: {url}")
            response = requests.get(url, headers=headers)
            soap = BeautifulSoup(response.content, "html.parser")
        except Exception as e:
            print(f"⚠️ Skipping fandom due to error: {e}")
            traceback.print_exc()  # This will show the stack trace
            continue
        
        fandom_items = soap.select("ol.alphabet.fandom.index.group li ul li a")
        for fandom in fandom_items:
            name = fandom.text.strip()
            encoded_tag = quote(name, safe="")
            href = f"https://archiveofourown.org/tags/{encoded_tag}"
            fandom_links.append({
                "Name": name,
                "URL": href,
                "Source URL": base_fandom_url
            })
        time.sleep(5)

    time.sleep(5)

fandom_df = pd.DataFrame(fandom_links)
fandom_df.to_csv('Fandom_links.csv', index=False, encoding='utf-8-sig')
print(f"✅ Total fandoms collected: {len(fandom_links)}")


Fetching fandoms from: https://archiveofourown.org/media/Anime%20*a*%20Manga/fandoms
Fetching fandoms from: https://archiveofourown.org/media/Books%20*a*%20Literature/fandoms
Fetching fandoms from: https://archiveofourown.org/media/Cartoons%20*a*%20Comics%20*a*%20Graphic%20Novels/fandoms
Fetching fandoms from: https://archiveofourown.org/media/Celebrities%20*a*%20Real%20People/fandoms
Fetching fandoms from: https://archiveofourown.org/media/Movies/fandoms
Fetching fandoms from: https://archiveofourown.org/media/Music%20*a*%20Bands/fandoms
Fetching fandoms from: https://archiveofourown.org/media/Other%20Media/fandoms
Fetching fandoms from: https://archiveofourown.org/media/Theater/fandoms
Fetching fandoms from: https://archiveofourown.org/media/TV%20Shows/fandoms
Fetching fandoms from: https://archiveofourown.org/media/Video%20Games/fandoms
Fetching fandoms from: https://archiveofourown.org/media/Uncategorized%20Fandoms/fandoms
✅ Total fandoms collected: 99060


In [39]:
fandom_df = pd.read_csv("Fandom_links.csv", encoding='utf-8')   
# ✅ Step 2: Scrape stories from each fandom (first page only)
story_data = []
batch_size = 300

batch_df = fandom_df.head(batch_size)  # Slice out 25 rows

for index, row in batch_df.iterrows():
    fandom_name = row["Name"]
    fandom_url = row["URL"]
    source_url = row["Source URL"]
    print(f"Scraping: {fandom_name}")
    
    stories = []
    try:
        works_url = fandom_url + "/works?page=1"
        print(f"Scraping stories from: {fandom_name}")
        print(works_url)
        
        response = requests.get(works_url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        stories = soup.find_all("li", class_="work")
    except Exception as e:
        print(f"⚠️ Skipping fandom due to error: {e}")
        traceback.print_exc()  # This will show the stack trace
        continue

    for story in stories:
        try:
            title_tag = story.find("h4", class_="heading")
            if title_tag:
                a_tag = title_tag.find('a')
                
                if a_tag and hasattr(a_tag, 'text'):
                    raw_title = a_tag.text.strip()
                    title = re.sub(r'\s+', ' ', raw_title)
                    story_url = 'https://archiveofourown.org' + a_tag['href']
                else:
                    title = "No title"
                    story_url = None
            else:
                title = "No title"
                story_url = None
                
            author_tag = story.find("a", rel="author")
            author = author_tag.text.strip() if author_tag else "Anonymous"
            summary = story.find("blockquote", class_="userstuff summary")
            summary_text = summary.text.strip() if summary else "No summary"
            stats = story.find("dl", class_="stats")
            kudos = stats.find("dd", class_="kudos").text.strip() if stats and stats.find("dd", class_="kudos") else "0"
            hits = stats.find("dd", class_="hits").text.strip() if stats and stats.find("dd", class_="hits") else "0"
            comments = stats.find("dd", class_="comments").text.strip() if stats and stats.find("dd", class_="comments") else "0"
            tags_block = story.find("ul", class_="tags")
            tags = [tag.text.strip() for tag in tags_block.find_all("li")] if tags_block else []

            story_data.append({
                "Title": title,
                "Author": author,
                "Summary": summary_text,
                "Kudos": kudos,
                "Hits": hits,
                "Comments": comments,
                "Tags": ", ".join(tags),
                "Fandom": fandom_name,
                "Source": "AO3", 
                "URL":story_url,
                "Parent URL": source_url
            })
            # Drop the used fandom row after successful scrape
            fandom_df.drop(labels=[index], inplace=True, errors='ignore')
        except Exception as e:
            print(f"⚠️ Skipping story due to error: {e}")
            traceback.print_exc()  # This will show the stack trace
        
        time.sleep(10)
        
fandom_df.to_csv('Fandom_links.csv', index=False, encoding='utf-8-sig')

# ✅ Step 3: Save to CSV
print("✅ Scraping complete!")


Scraping: 12歳。～ちっちゃなムネのトキメキ～ | 12-sai.: Chicchana Mune no Tokimeki (Anime)
Scraping stories from: 12歳。～ちっちゃなムネのトキメキ～ | 12-sai.: Chicchana Mune no Tokimeki (Anime)
https://archiveofourown.org/tags/12%E6%AD%B3%E3%80%82%EF%BD%9E%E3%81%A1%E3%81%A3%E3%81%A1%E3%82%83%E3%81%AA%E3%83%A0%E3%83%8D%E3%81%AE%E3%83%88%E3%82%AD%E3%83%A1%E3%82%AD%EF%BD%9E%20%7C%2012-sai.%3A%20Chicchana%20Mune%20no%20Tokimeki%20%28Anime%29/works?page=1
Scraping: 2.43 清陰高校男子バレー部 | 2.43: Seiin Koukou Danshi Volley-bu (Anime)
Scraping stories from: 2.43 清陰高校男子バレー部 | 2.43: Seiin Koukou Danshi Volley-bu (Anime)
https://archiveofourown.org/tags/2.43%20%E6%B8%85%E9%99%B0%E9%AB%98%E6%A0%A1%E7%94%B7%E5%AD%90%E3%83%90%E3%83%AC%E3%83%BC%E9%83%A8%20%7C%202.43%3A%20Seiin%20Koukou%20Danshi%20Volley-bu%20%28Anime%29/works?page=1
Scraping: 2.5次元の誘惑 | 2.5-jigen no Ririsa | 2.5 Dimensional Seduction (Anime)
Scraping stories from: 2.5次元の誘惑 | 2.5-jigen no Ririsa | 2.5 Dimensional Seduction (Anime)
https://archiveofourown.org/tags/2.5%E6%

Traceback (most recent call last):
  File "c:\Users\babym\anaconda3\Lib\site-packages\urllib3\connection.py", line 196, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\babym\anaconda3\Lib\site-packages\urllib3\util\connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\babym\anaconda3\Lib\socket.py", line 964, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
socket.gaierror: [Errno 11001] getaddrinfo failed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\babym\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 789, in urlopen
    response = self._make_request(
          

Scraping: 映像研には手を出すな! | Eizouken ni wa Te wo Dasu na! | Keep Your Hands off Eizoken! (Anime)
Scraping stories from: 映像研には手を出すな! | Eizouken ni wa Te wo Dasu na! | Keep Your Hands off Eizoken! (Anime)
https://archiveofourown.org/tags/%E6%98%A0%E5%83%8F%E7%A0%94%E3%81%AB%E3%81%AF%E6%89%8B%E3%82%92%E5%87%BA%E3%81%99%E3%81%AA%21%20%7C%20Eizouken%20ni%20wa%20Te%20wo%20Dasu%20na%21%20%7C%20Keep%20Your%20Hands%20off%20Eizoken%21%20%28Anime%29/works?page=1
⚠️ Skipping fandom due to error: HTTPSConnectionPool(host='archiveofourown.org', port=443): Max retries exceeded with url: /tags/%E6%98%A0%E5%83%8F%E7%A0%94%E3%81%AB%E3%81%AF%E6%89%8B%E3%82%92%E5%87%BA%E3%81%99%E3%81%AA%21%20%7C%20Eizouken%20ni%20wa%20Te%20wo%20Dasu%20na%21%20%7C%20Keep%20Your%20Hands%20off%20Eizoken%21%20%28Anime%29/works?page=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000018457761CD0>: Failed to resolve 'archiveofourown.org' ([Errno 11001] getaddrinfo failed)"))
Scraping: エル・カザド | El 

Traceback (most recent call last):
  File "c:\Users\babym\anaconda3\Lib\site-packages\urllib3\connection.py", line 196, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\babym\anaconda3\Lib\site-packages\urllib3\util\connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\babym\anaconda3\Lib\socket.py", line 964, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
socket.gaierror: [Errno 11001] getaddrinfo failed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\babym\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 789, in urlopen
    response = self._make_request(
          

Scraping: Endride (Anime)
Scraping stories from: Endride (Anime)
https://archiveofourown.org/tags/Endride%20%28Anime%29/works?page=1
⚠️ Skipping fandom due to error: HTTPSConnectionPool(host='archiveofourown.org', port=443): Max retries exceeded with url: /tags/Endride%20%28Anime%29/works?page=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000018454BBEC90>: Failed to resolve 'archiveofourown.org' ([Errno 11001] getaddrinfo failed)"))
Scraping: えんどろ～！ | Endro! (Anime)
Scraping stories from: えんどろ～！ | Endro! (Anime)
https://archiveofourown.org/tags/%E3%81%88%E3%82%93%E3%81%A9%E3%82%8D%EF%BD%9E%EF%BC%81%20%7C%20Endro%21%20%28Anime%29/works?page=1
⚠️ Skipping fandom due to error: HTTPSConnectionPool(host='archiveofourown.org', port=443): Max retries exceeded with url: /tags/%E3%81%88%E3%82%93%E3%81%A9%E3%82%8D%EF%BD%9E%EF%BC%81%20%7C%20Endro%21%20%28Anime%29/works?page=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000

...

⚠️ Skipping fandom due to error: HTTPSConnectionPool(host='archiveofourown.org', port=443): Max retries exceeded with url: /tags/Fangs%20%28Manga%29/works?page=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000018457D7BE30>: Failed to resolve 'archiveofourown.org' ([Errno 11001] getaddrinfo failed)"))
Scraping: Fantastic Children
Scraping stories from: Fantastic Children
https://archiveofourown.org/tags/Fantastic%20Children/works?page=1
⚠️ Skipping fandom due to error: HTTPSConnectionPool(host='archiveofourown.org', port=443): Max retries exceeded with url: /tags/Fantastic%20Children/works?page=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000018452EC1160>: Failed to resolve 'archiveofourown.org' ([Errno 11001] getaddrinfo failed)"))
Scraping: 異世界美少女受肉おじさんと | Fantasy Bishoujo Juniku Ojisan to | Total Fantasy Knockout (Anime)
Scraping stories from: 異世界美少女受肉おじさんと | Fantasy Bishoujo Juniku Ojisan to | Total Fantasy K

Traceback (most recent call last):
  File "c:\Users\babym\anaconda3\Lib\site-packages\urllib3\connection.py", line 196, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\babym\anaconda3\Lib\site-packages\urllib3\util\connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\babym\anaconda3\Lib\socket.py", line 964, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
socket.gaierror: [Errno 11001] getaddrinfo failed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\babym\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 789, in urlopen
    response = self._make_request(
          

Scraping: Fate/type Redline
Scraping stories from: Fate/type Redline
https://archiveofourown.org/tags/Fate%2Ftype%20Redline/works?page=1
⚠️ Skipping fandom due to error: HTTPSConnectionPool(host='archiveofourown.org', port=443): Max retries exceeded with url: /tags/Fate%2Ftype%20Redline/works?page=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000184575B3500>: Failed to resolve 'archiveofourown.org' ([Errno 11001] getaddrinfo failed)"))
Scraping: Fate/Zero
Scraping stories from: Fate/Zero
https://archiveofourown.org/tags/Fate%2FZero/works?page=1
⚠️ Skipping fandom due to error: HTTPSConnectionPool(host='archiveofourown.org', port=443): Max retries exceeded with url: /tags/Fate%2FZero/works?page=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000018452AB76E0>: Failed to resolve 'archiveofourown.org' ([Errno 11001] getaddrinfo failed)"))
Scraping: フォスティーヌ | Faustine (Manga)
Scraping stories from: フォスティーヌ | Faustine (

Traceback (most recent call last):
  File "c:\Users\babym\anaconda3\Lib\site-packages\urllib3\connection.py", line 196, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\babym\anaconda3\Lib\site-packages\urllib3\util\connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\babym\anaconda3\Lib\socket.py", line 964, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
socket.gaierror: [Errno 11001] getaddrinfo failed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\babym\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 789, in urlopen
    response = self._make_request(
          

Scraping: Franken Fran
Scraping stories from: Franken Fran
https://archiveofourown.org/tags/Franken%20Fran/works?page=1
⚠️ Skipping fandom due to error: HTTPSConnectionPool(host='archiveofourown.org', port=443): Max retries exceeded with url: /tags/Franken%20Fran/works?page=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001845BC59F70>: Failed to resolve 'archiveofourown.org' ([Errno 11001] getaddrinfo failed)"))
Scraping: Frau Faust (Manga)
Scraping stories from: Frau Faust (Manga)
https://archiveofourown.org/tags/Frau%20Faust%20%28Manga%29/works?page=1
⚠️ Skipping fandom due to error: HTTPSConnectionPool(host='archiveofourown.org', port=443): Max retries exceeded with url: /tags/Frau%20Faust%20%28Manga%29/works?page=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000018452C51A60>: Failed to resolve 'archiveofourown.org' ([Errno 11001] getaddrinfo failed)"))
Scraping: Free Soul (Manga)
Scraping stories from: Free 

In [42]:
csv_data = pd.read_csv(fileName, encoding='utf-8')   
csv_data_1 = pd.read_csv("ao3_books_fandom_stories - Copy.csv")
df = pd.DataFrame(story_data)
print(len(story_data),len(csv_data), len(df), len(csv_data_1))
links = pd.read_csv("Fandom_links.csv", encoding='utf_8_sig') 
print(len(links))

1156 17906 1156 12345
96694


In [41]:
combined_df = pd.concat([df, csv_data], ignore_index=True)
combined_df.to_csv(fileName, index=False, encoding='utf_8_sig')
len(combined_df)

17906

In [12]:
df.to_csv(fileName, index=False, encoding='utf_8_sig')

### Not in use

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {
    "User-Agent": "Mozilla/5.0",
}
base_url = "https://archiveofourown.org/works/search?work_search[query]=romance&page={}"

story_data = []
comments_data = []

for page in range(1, 4):  # Scrape first 3 pages
    url = base_url.format(page)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    stories = soup.find_all("li", class_="work")

    for story in stories:
        # Title & Author
        title_tag = story.find("h4", class_="heading")
        title = title_tag.text.strip() if title_tag else "No Title"
        link = "https://archiveofourown.org" + title_tag.find("a")["href"] if title_tag and title_tag.find("a") else "N/A"
        
        author_tag = story.find("a", rel="author")
        author = author_tag.text.strip() if author_tag else "Anonymous"

        # Summary
        summary_tag = story.find("blockquote", class_="userstuff summary")
        summary = summary_tag.text.strip() if summary_tag else "No summary"

        # Stats
        stats = story.find("dl", class_="stats")
        kudos = stats.find("dd", class_="kudos").text.strip() if stats and stats.find("dd", class_="kudos") else "0"
        hits = stats.find("dd", class_="hits").text.strip() if stats and stats.find("dd", class_="hits") else "0"
        comments_count = stats.find("dd", class_="comments").text.strip() if stats and stats.find("dd", class_="comments") else "0"

        # Tags
        tags_block = story.find("ul", class_="tags")
        tags = [tag.text.strip() for tag in tags_block.find_all("li")] if tags_block else []

        story_data.append({
            "Title": title,
            "Author": author,
            "Link": link,
            "Summary": summary,
            "Kudos": kudos,
            "Comments Count": comments_count,
            "Hits": hits,
            "Tags": ", ".join(tags)
        })

        # Extract Comments (Optional and slower)
        if link != "N/A":
            try:
                story_page = requests.get(link, headers=headers)
                story_soup = BeautifulSoup(story_page.content, "html.parser")

                comment_blocks = story_soup.select("ol.thread li.comment")
                for comment in comment_blocks:
                    commenter = comment.find("a", class_="comment_user")
                    commenter_name = commenter.text.strip() if commenter else "Guest"

                    comment_body = comment.find("blockquote", class_="userstuff")
                    comment_text = comment_body.text.strip() if comment_body else "No comment text"

                    comments_data.append({
                        "Story Title": title,
                        "Author": author,
                        "Commenter": commenter_name,
                        "Comment": comment_text
                    })

                time.sleep(2)  # Be gentle
            except Exception as e:
                print(f"Error fetching comments for {title}: {e}")

    time.sleep(5)  # Be respectful, don’t spam requests

# Save to Excel with two sheets

pd.DataFrame(story_data).to_csv("ao3_romance_stories.csv", index=False)
pd.DataFrame(comments_data).to_csv("ao3_romance_comments.csv", index=False)

print("✅ Scraping complete! Data saved to ao3_romance_stories.xlsx")


✅ Scraping complete! Data saved to ao3_sample_stories.csv


In [15]:
import requests
from bs4 import BeautifulSoup
import time
import re

base_url = "https://www.fanfiction.net/j/0/0/0/"

headers = {
    "User-Agent": "Chrome/122.0.0.0 Safari/537.36"
}

for page in range(1, 4):  # First 3 pages
    url = base_url.format(page)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    stories = soup.find_all("div", class_="z-list")

    for story in stories:
        title = story.find("a", class_="stitle").text
        summary = story.find("div", class_="z-indent z-padtop").text.strip()

        meta = story.find("div", class_="z-indent z-padtop2 xgray").text
        reviews = re.search(r"Reviews: (\d+)", meta)
        favs = re.search(r"Favs: (\d+)", meta)
        follows = re.search(r"Follows: (\d+)", meta)

        reviews = reviews.group(1) if reviews else "0"
        favs = favs.group(1) if favs else "0"
        follows = follows.group(1) if follows else "0"

        print(f"Reviews: {reviews}, Favs: {favs}, Follows: {follows}")

        genre_line = story.find("div", class_="z-indent z-padtop2 xgray").text
        genre_match = re.search(r"- ([A-Za-z/]+) -", genre_line)
        genre = genre_match.group(1) if genre_match else "Unknown"
        print("Genre:", genre)

        print(f"Title: {title}\nSummary: {summary}\n")

    time.sleep(5)
